In [1]:
import cfgmgr.config_manager as cfg
import eii.msgbus as mb
import threading
import sys
import time
import ipynb.fs.full.udf_template as notebook_udf

In [3]:
def main():
    """Main method.
    """
    # Initializing Etcd to set env variables
    ctx = cfg.ConfigMgr()
    dev_mode = ctx.is_dev_mode()
    if dev_mode is False:
        print("PROD mode not supported, please re-run in DEV mode.")
        sys.exit(1)

    msgbus_cfg = {
                    "type": "zmq_ipc",
                    "socket_dir": "/EII/sockets"
                 }

    # Initializing msgbus for subscribing
    msgbus = mb.MsgbusContext(msgbus_cfg)
    subscriber = msgbus.new_subscriber("jupyter_publisher")
        
    # Initializing msgbus for publishing
    pub_msgbus = mb.MsgbusContext(msgbus_cfg)
    publisher = pub_msgbus.new_publisher("jupyter_subscriber")

    # Receive UDF config from jupyter_connector
    conf_sub = msgbus.new_subscriber("jupyter_config")
    udf_config, _ = conf_sub.recv()

    custom_udf = notebook_udf.Udf(udf_config)
    while True:
        metadata, blob = subscriber.recv()
        frame_drop, frame_updated, md = custom_udf.process(blob, metadata)
        if frame_drop is True:
            print("Frame is skipped in UDF processs method")
            metadata["jpnb_frame_drop"] = True
        elif frame_updated is not None and md is not None:
            metadata["jpnb_frame_updated"] = True
            metadata["jpnb_metadata_updated"] = True
            publisher.publish((md, frame_updated.tobytes(),))
        elif frame_updated is not None or md is not None:
            if frame_updated is not None:
                print("Frame is updated in UDF processs method")
                metadata["jpnb_frame_updated"] = True
                publisher.publish((metadata, frame_updated.tobytes(),))
            if md is not None:
                print("metadata is updated in UDF processs method")
                metadata["jpnb_metadata_updated"] = True
                publisher.publish((md, blob,))
        else:
            publisher.publish((metadata, blob,))

if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        raise("Exception {}".format(e))

{'w': 1920.0, 'h': 1200.0}
(True, None, None)
